In [1]:
import torch
import torchvision
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device=torch.device("mps")
#DataNormalization

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

classes=("0","1","2","3","4","5","6","7","8","9")
class NN(nn.Module):
    def __init__(self):
        super(NN,self).__init__()
        self.cn1=nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,padding=1)
        self.cn2=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2=nn.Linear(128,64)
        self.out=nn.Linear(64,10)
    def forward(self,x):
        x=self.pool(F.relu(self.cn1(x)))
        x=self.pool(F.relu(self.cn2(x)))
        x=x.view(-1,64*7*7)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.out(x)
        return x
model=NN().to(device)
#Adding hyperparameters
lr=3e-4
epochs=10
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=lr)
for epoch in range(epochs):
    for i,(images,labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        out=model.forward(images)
        loss=loss_fn(out,labels)
        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')
with torch.no_grad():
    total,correct=0,0
    for images,test_labels in test_loader:
        images, test_labels = images.to(device), test_labels.to(device)
        out1=model(images)
        _, predicted = torch.max(out1.data, 1)
        total += test_labels.size(0)
        correct += (predicted == test_labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy:.2f}%')      

Epoch [1/10], Step [100/938], Loss: 0.4204
Epoch [1/10], Step [200/938], Loss: 0.1318
Epoch [1/10], Step [300/938], Loss: 0.1653
Epoch [1/10], Step [400/938], Loss: 0.0952
Epoch [1/10], Step [500/938], Loss: 0.1090
Epoch [1/10], Step [600/938], Loss: 0.1553
Epoch [1/10], Step [700/938], Loss: 0.0883
Epoch [1/10], Step [800/938], Loss: 0.0352
Epoch [1/10], Step [900/938], Loss: 0.1128
Epoch [2/10], Step [100/938], Loss: 0.0416
Epoch [2/10], Step [200/938], Loss: 0.1889
Epoch [2/10], Step [300/938], Loss: 0.0464
Epoch [2/10], Step [400/938], Loss: 0.2115
Epoch [2/10], Step [500/938], Loss: 0.0337
Epoch [2/10], Step [600/938], Loss: 0.0282
Epoch [2/10], Step [700/938], Loss: 0.1032
Epoch [2/10], Step [800/938], Loss: 0.0313
Epoch [2/10], Step [900/938], Loss: 0.0243
Epoch [3/10], Step [100/938], Loss: 0.0861
Epoch [3/10], Step [200/938], Loss: 0.0582
Epoch [3/10], Step [300/938], Loss: 0.0513
Epoch [3/10], Step [400/938], Loss: 0.1002
Epoch [3/10], Step [500/938], Loss: 0.0467
Epoch [3/10